### Definicja poniższej funkcji z wykorzystaniem `sympy`

```python
def r(t_):
    return -theta_1*lambda_1*m.exp(-lambda_1*t_) - theta_2*lambda_2*m.exp(-lambda_2*t_) - theta_3*k3*m.exp(-k3*t_)
```

In [1]:
import sympy as sp

In [2]:
# definicja symboli
theta_1, theta_2, theta_3 = sp.symbols('\\theta_1 \\theta_2 \\theta_3')
lambda_1, lambda_2 = sp.symbols('\lambda_1 \lambda_2')
t_, k3 = sp.symbols('t k_3')

# definicja wyrażenia r skopiowana ze skryptu ze zmianą m -> sp
r = -theta_1*lambda_1*sp.exp(-lambda_1*t_) - theta_2*lambda_2*sp.exp(-lambda_2*t_) - theta_3*k3*sp.exp(-k3*t_)

In [3]:
# wyświetl wyrażenie
r

-\lambda_1*\theta_1*exp(-\lambda_1*t) - \lambda_2*\theta_2*exp(-\lambda_2*t) - \theta_3*k_3*exp(-k_3*t)

Powyższa forma zdecydowania ułatwia "odnalezienie" się w implementowanym artykule i znalezienie ewentualnych błędów.

Jeżeli chcemy skorzystać z wyrażenia 'r' jako funkcji zależnej od czasu, można wykorzystać `lambdify`:

In [4]:
r_ = sp.lambdify((theta_1,theta_2,theta_3,lambda_1,lambda_2,k3,t_),r)

# teraz r_ jest funkcją która przyjmuje 7 parametrów:
print(r_(1,2,3,4,5,6,7))

-2.7720755086763038e-12


Bardziej czytelny sposób, wykorzystanie `subs` do podstawiania parametrów stałych:

In [5]:
# definicja "pierwszej porcji" parametrów
params = {theta_1:1,theta_2:2,theta_3:3} 
# rozszerzenie zmiennej `params`
params.update({lambda_1:4,lambda_2:5,k3:6})
# definicja `r_` jako funkcji zależnej od tylko od `t_`
r_ = sp.lambdify((t_),r.subs(params))

# teraz r_ jest funkcją o parametrach zdefiniowanych w params,
# przyjmująca jeden argument: t_
print(r_(7))

-2.772075508676304e-12


## Definicja symboli zależnych:

In [6]:
# definicja zmiennej niezależnej
t = sp.symbols('t') 

# definicja symboli typu 'funkcja', czyli zależnych
# w cudzysłowach wykorzystywana jest notacja latexa,
# podwójny '\' przed 'theta' ponieważ '\t' jest znakiem specjalnym
phi, tta = sp.symbols('\phi \\theta', cls=sp.Function) 

# definicja symboli niezależnych (np. niezmiennych w czasie)
x,y = sp.symbols('x,y') 

# definicja wyrażenia f1
f1 = x**2+phi(t)+tta(t)**3*y 

# pochodna wyrażenia f1 po t
df1 = sp.diff(f1,t) 

In [7]:
phi(t)

\phi(t)

In [8]:
tta(t)

\theta(t)

In [9]:
f1

x**2 + y*\theta(t)**3 + \phi(t)

In [10]:
df1

3*y*\theta(t)**2*Derivative(\theta(t), t) + Derivative(\phi(t), t)

# 25 maja 2020

## Suma dwóch funkcji

In [11]:
f2 = df1+f1
f2

x**2 + y*\theta(t)**3 + 3*y*\theta(t)**2*Derivative(\theta(t), t) + \phi(t) + Derivative(\phi(t), t)

## Podstawienie pod funkcję innej zmiennej

In [12]:
f2.subs({tta(t):x})

x**3*y + 3*x**2*y*Derivative(x, t) + x**2 + \phi(t) + Derivative(\phi(t), t)

## Podstawienie pod funkcje innej funkcji

In [13]:
f2.subs({tta(t):phi(t)})

x**2 + y*\phi(t)**3 + 3*y*\phi(t)**2*Derivative(\phi(t), t) + \phi(t) + Derivative(\phi(t), t)

## Zmiana argumentu funkcji

In [14]:
f2.subs({t:x})

x**2 + y*\theta(x)**3 + 3*y*\theta(x)**2*Derivative(\theta(x), x) + \phi(x) + Derivative(\phi(x), x)

# Propozycja jak implementować złożone wyrażenia np.:
$$
f_1 = a \cdot c \\
a = d(t) + b(t) \\
b = d(t)^2+c(t) \\
c = d(t) \cdot 10 \\
d = t \cdot 5
$$
Pomimo tego, że $a$, $b$ i $c$ w pierwszym równaniu nie są zapisane w formie funkcji zależnej od czasu (czyli np. $a(t)$). Kolejne równania sugerują, że jest to jednak tylko konwencja dla zachowania czytelności zapisu.

## Krok 1: implementacja wyrażenia głównego

In [15]:
t = sp.symbols('t')
a, c = sp.symbols('a c',cls=sp.Function) # deklaracja zmiennych
f_1 = a(t) * c(t) # deklaracja funkcji
display(f_1) # wyświetlenie funkcji w celu jej weryfikacji ze wzorem

a(t)*c(t)

## Krok 2: implementacja wyrażeń "wewnętrznych"

In [16]:
b, d = sp.symbols('b d', cls=sp.Function)
a_ = d(t) + b(t) # znak '_' jest wykorzystany, aby oznaczyć "tymczasowość" zmiennej, może tutaj być cokolwiek
podstaw = {a(t):a_} # zmienna podstaw zostanie wykorzystana "na koniec"
display(a_) # w tym miejscu najlepiej podać numer równania z artykułu

b(t) + d(t)

In [17]:
b_ = d(t)**2 + c(t)
podstaw.update({b(t):b_})
display(b_)

c(t) + d(t)**2

In [18]:
c_ = d(t)*10
podstaw.update({c(t):c_})
display(c_)

10*d(t)

In [19]:
d_ = t*5
podstaw.update({d(t):d_})
display(d_)

5*t

### Jeżeli wymagana jest kolejna "seria" podstawień, warto zastosować notację "_n", gdzie "n" jest kolejnym podstawieniem

## Krok 3: weryfikacja poprawności poszczególnych części wyrażenia.
Zwłaszcza dla bardziej skomplikowanych wyrażeń pozwoli na zebranie wszystkiego w jednym miejscu (ułatwi ewentualną weryfikację), ale co ważniejsze, pozwoli potwierdzić, że nie został popełniony jakiś drobny błąd, np. nadpisanie zmiennej.

In [20]:
display(f_1,a_,b_,c_,d_) # i tutaj kolejne numery równań

a(t)*c(t)

b(t) + d(t)

c(t) + d(t)**2

10*d(t)

5*t

## Krok 4: podstawienie zmiennych i zamiana wyrażenia symbolicznego na numeryczne.

In [21]:
display(f_1.subs(podstaw)) # nie jest to niezbędny krok i rozumiem, że wynik mógł być nieczytelny dla skomplikowanych wyrażeń.
f_1_num = sp.lambdify((t),f_1.subs(podstaw))

50*t*(25*t**2 + 55*t)

# Krok 5: testy
W związku z dosyć "napiętym grafikiem", krok ten jest opcjonalny, jednak myślę, że warto się nad nim zastanowić, gdyż pozwala na zautomatyzowanie weryfikacji poprawności implementacji. Ewentualnie ułatwienie wykrywania błędów w równaniach.

Przykładowo, jeżeli jesteśmy wstanie określić (np. obliczyć "ręcznie") jaki powinien być wynik danego wyrażenia dla pewnych wyjątkowych przypadków, lub chociażby rozmiar (w przypadku macierzy). Możemy wykorzystać metody automatycznego testowania. W pythonie, najbardziej podstawowym narzędziem jest moduł [`doctest`](https://docs.python.org/3.8/library/doctest.html).

Dla powyższego przypadku mogłoby to wyglądać np.:

In [22]:
import doctest

def przykladoweTesty(t):
    """
    Funkcja testująca poprawność implementacji.
    
    >>> f_1_num(0)
    0
    >>> f_1_num(1)
    10
    >>> f_1_num(1) < 5000
    True
    """

doctest.testmod()    

**********************************************************************
File "__main__", line 9, in __main__.przykladoweTesty
Failed example:
    f_1_num(1)
Expected:
    10
Got:
    4000
**********************************************************************
1 items had failures:
   1 of   3 in __main__.przykladoweTesty
***Test Failed*** 1 failures.


TestResults(failed=1, attempted=3)

W powyższym przykładie jeden test zakończył się niepomyślnie. Po zamianie w drugim teście `10` na `4000`, wszystkie testy powinny się skończyć pomyślnie.